# Load data

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.stats import pearsonr
 
    
df_panel = pd.read_csv('panel_data_all_with_flu_rate.csv')
df_merged = pd.read_csv('df_merged.csv')
# df_merged = df_merged[df_merged['avg_users'] > 1000].reset_index(drop=True) #22
df_merged = df_merged[df_merged['user_rate'] > 0.5].reset_index(drop=True) #37
ciyt_list = df_merged['Country-city'].tolist()
print(ciyt_list)

# 强制转为数值，清理脏数据
df_panel['pm25'] = pd.to_numeric(df_panel['pm25'], errors='coerce')
df_panel['cough_rate'] = pd.to_numeric(df_panel['cough_rate'], errors='coerce')

# 删除缺失值
df_panel = df_panel.dropna(subset=['cough_rate', 'pm25', 'city', 'country'])

# 转换为 datetime 类型（必须）
df_panel['date'] = pd.to_datetime(df_panel['date'], errors='coerce')
# 删除指定日期区间的数据
df_panel = df_panel[~((df_panel['date'] >= "2024-04-20") & (df_panel['date'] <= "2024-04-23"))]
df_panel['label'] = df_panel['country'] + '-' + df_panel['city']

# 提取城市
cities =  df_panel['label'].tolist()
ciyt_list = list(set(ciyt_list) & set(cities))
ciyt_list = sorted(ciyt_list)
print(len(ciyt_list))

df_panel = df_panel[df_panel['label'].isin(ciyt_list)]

['Australia-Brisbane', 'Australia-Melbourne', 'Australia-Sydney', 'Brazil-Sao Paulo', 'Chile-Santiago', 'China-Beijing', 'China-Chengdu', 'China-Hangzhou', 'China-Shanghai', 'France-Paris', 'Germany-Berlin', 'Germany-Leipzig', 'Italy-Bergamo', 'Italy-Florence', 'Italy-Milano', 'Italy-Monza and Brianza', 'Italy-Rome', 'Japan-Osaka', 'Japan-Tokyo', 'South Africa-City of Cape Town', 'South Africa-City of Johannesburg', 'Sweden-Uppsala', 'United Kingdom-Birmingham', 'United Kingdom-Cambridgeshire', 'United Kingdom-London', 'United Kingdom-Oxfordshire', 'United States-Chicago', 'United States-Columbus', 'United States-Los Angeles', 'United States-New York', 'United States-Phoenix', 'United States-Pittsburgh', 'United States-Saint Louis', 'United States-Salt Lake']
34


In [2]:
df_panel

,cough_rate,date,pm25,city,country,TAVG,PRCP,flu_rate,label
933,0.607586,2023-10-01,39.0,Beijing,China,62.0,0.00,0.036884,China-Beijing
934,0.699118,2023-10-02,94.0,Beijing,China,62.0,0.00,0.044940,China-Beijing
935,0.659417,2023-10-03,106.0,Beijing,China,67.0,0.08,0.044940,China-Beijing
936,0.555569,2023-10-04,32.0,Beijing,China,60.0,0.00,0.044940,China-Beijing
937,0.597655,2023-10-05,35.0,Beijing,China,59.0,0.00,0.044940,China-Beijing
...,...,...,...,...,...,...,...,...,...
22935,0.530524,2025-02-24,42.0,Sao Paulo,Brazil,77.0,0.00,0.025316,Brazil-Sao Paulo
22936,0.521488,2025-02-25,49.0,Sao Paulo,Brazil,78.0,0.00,0.025316,Brazil-Sao Paulo
22937,0.517456,2025-02-26,49.0,Sao Paulo,Brazil,78.0,0.00,0.025316,Brazil-Sao Paulo
22938,0.509430,2025-02-27,58.0,Sao Paulo,Brazil,78.0,0.00,0.025316,Brazil-Sao Paulo


# Selected Lag

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress, t
import numpy as np

import pandas as pd
from linearmodels.panel import PanelOLS
import statsmodels.api as sm

# ==== 1) 准备数据 ====
df = df_panel.copy()

# 确保类型正确
df["pm25"] = pd.to_numeric(df["pm25"], errors="coerce")
df["TAVG"] = pd.to_numeric(df["TAVG"], errors="coerce")
df["PRCP"] = pd.to_numeric(df["PRCP"], errors="coerce")
df["flu_rate"] = pd.to_numeric(df["flu_rate"], errors="coerce")
df["cough_rate"] = pd.to_numeric(df["cough_rate"], errors="coerce")

# 设定面板索引并排序（非常重要：生成滞后前要按 city、date 排序）
df = df.set_index(["city", "date"]).sort_index()

# 生成 pm25 的滞后1~3天（在每个 city 内部做 shift）
for k in [1, 2, 3, 4, 5]:
    df[f"pm25_l{k}"] = df.groupby(level="city")["pm25"].shift(k)
 
df["pm25_flu"] = df["pm25"] * df["flu_rate"]
df["cough_rate_log"] = np.log(df["cough_rate"])

# 丢弃缺失（滞后会产生NaN）
exog_vars = ["pm25","flu_rate", "TAVG", "PRCP"] #, "pm25_flu"]
df = df.dropna(subset=exog_vars + ["cough_rate"])

# 解释变量与因变量
X = df[exog_vars]
y = df["cough_rate_log"]

# ==== 2) 固定效应面板回归 ====
# 同时加入 city 和 time 固定效应；drop_absorbed=True 防止完全共线变量报错
mod = PanelOLS(y, X, entity_effects=True, time_effects=True, drop_absorbed=True)

# 聚类稳健标准误：对实体与时间同时聚类（推荐）
res = mod.fit(cov_type="clustered", cluster_entity=True, cluster_time=True)

print(res.summary)



                          PanelOLS Estimation Summary                           
Dep. Variable:         cough_rate_log   R-squared:                        0.2459
Estimator:                   PanelOLS   R-squared (Between):              0.8017
No. Observations:               15297   R-squared (Within):               0.3148
Date:                Sat, Nov 29 2025   R-squared (Overall):              0.7336
Time:                        12:05:31   Log-likelihood                    8687.9
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      1202.1
Entities:                          34   P-value                           0.0000
Avg Obs:                       449.91   Distribution:                 F(4,14750)
Min Obs:                       223.00                                           
Max Obs:                       507.00   F-statistic (robust):             22.713
                            

In [5]:
import pandas as pd
import numpy as np
from linearmodels.panel import PanelOLS
import statsmodels.api as sm

# ==== 1) 准备数据 ====
df = df_panel.copy()

# 确保类型正确
for col in ["pm25","TAVG","PRCP","flu_rate","cough_rate"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# 设定面板索引并排序（按 city、date）
df = df.set_index(["city", "date"]).sort_index()

# 生成 pm25 的滞后1~3天（在每个 city 内部做 shift）
for k in [1, 2, 3, 4, 5]:
    df[f"pm25_l{k}"] = df.groupby(level="city")["pm25"].shift(k)
    
# ---- 关键步骤：构造 Flu anomaly（去季节化）----
# 从 MultiIndex 里取出日期的月份
month = df.index.get_level_values("date").month
df["month"] = month

# 城市×月份的基线（用中位数或均值都可；中位数更稳健）
df["flu_base"] = df.groupby(["city","month"])["flu_rate"].transform("median")
df["flu_anom"] = df["flu_rate"] - df["flu_base"]   # 去季节化后的流感异常量

# 交互项改为：pm25 × flu_anom
df["pm25_fluanom"] = df["pm25"] * df["flu_anom"]
df["pm25L1_fluanom"] = df["pm25_l1"] * df["flu_anom"]
df["pm25L2_fluanom"] = df["pm25_l2"] * df["flu_anom"]

# 因变量对数化（避免 log(0)）
df["cough_rate_log"] = np.log(df["cough_rate"].clip(lower=1e-6))


# ==== 2) 丢弃缺失 ====
exog_vars = ["pm25", "flu_anom", "TAVG", "PRCP", "pm25_fluanom"]
df = df.dropna(subset=exog_vars + ["cough_rate_log"])

# 解释变量与因变量
X = df[exog_vars]
y = df["cough_rate_log"]

# ==== 3) 固定效应面板回归 ====
# A) 先保持你原来的「每日 time FE」：能最强控制共性时间冲击
#    （注意：每日FE很强，若吸收过多波动，可改用‘年月FE’，见下方B方案）
mod = PanelOLS(y, X, entity_effects=True, time_effects=True, drop_absorbed=True)
res = mod.fit(cov_type="clustered", cluster_entity=True, cluster_time=True)
print(res.summary)



# ==== 2) 丢弃缺失 ====
exog_vars = ["pm25", "flu_anom", "TAVG", "PRCP", "pm25_fluanom", "pm25L1_fluanom", "pm25L2_fluanom"]
df = df.dropna(subset=exog_vars + ["cough_rate_log"])

# 解释变量与因变量
X = df[exog_vars]
y = df["cough_rate_log"]

# ==== 3) 固定效应面板回归 ====
# A) 先保持你原来的「每日 time FE」：能最强控制共性时间冲击
#    （注意：每日FE很强，若吸收过多波动，可改用‘年月FE’，见下方B方案）
mod = PanelOLS(y, X, entity_effects=True, time_effects=True, drop_absorbed=True)
res = mod.fit(cov_type="clustered", cluster_entity=True, cluster_time=True)
print(res.summary)



                          PanelOLS Estimation Summary                           
Dep. Variable:         cough_rate_log   R-squared:                        0.2346
Estimator:                   PanelOLS   R-squared (Between):              0.8757
No. Observations:               15297   R-squared (Within):               0.3147
Date:                Sat, Nov 29 2025   R-squared (Overall):              0.8020
Time:                        12:05:59   Log-likelihood                    8575.0
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      904.27
Entities:                          34   P-value                           0.0000
Avg Obs:                       449.91   Distribution:                 F(5,14749)
Min Obs:                       223.00                                           
Max Obs:                       507.00   F-statistic (robust):             22.767
                            